# Projet 9 :  Application mobile de recommandation de contenu

<font color = "grey"><i> Chargement des modules necessaires </i></font>

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import warnings
from surprise import Reader, Dataset
from surprise.model_selection import GridSearchCV
from surprise.model_selection import train_test_split
from surprise import SVD, accuracy
from surprise.model_selection import cross_validate
#from sklearn.model_selection import train_test_split

import _pickle as cPickle
import pickle
from random import randint
from scipy.spatial import distance

warnings.filterwarnings('ignore')

sns.set()
plt.style.use('ggplot')


# Sommaire
## I) Présentation générale
### Taille du jeu de données
###  Entrevue du jeu de données
### Types de données et leurs effectif

## II) Content-Based Filtering Model
### Presentation

## III) Collaborative Filtering Model
### Presentation

Légende:

><font color = "grey"><i> Je suis une petite indication </i></font>

><font color = "cyan"><i> Je suis une indication </i></font>

><font color = "green"><i> Je suis un commentaire </i></font>

<font color = "grey"><i> Chargement du style </i></font>

In [2]:
plt.style.use('ggplot')

## I) Présentation générale

![img](data/sys_rec.JPG )

<font color = "cyan"><i>  Chargement des jeux de données</i></font>

In [3]:
missing_values = ["na", "n/a", "--", " "]
df_articles_meta = pd.read_csv("data/articles_metadata.csv", sep=',', na_values=missing_values, low_memory=False)

df_cliks = pd.read_csv("data/clicks.csv", sep=',', na_values=missing_values, low_memory=False)

df_cliks_samp = pd.read_csv("data/clicks_sample.csv", sep=',', na_values=missing_values,  low_memory=False)

<a id='data_size'></a>
### I.2) Taille du jeu de données

<font color = "cyan"><i> Apercu des permières lignes et colonnes de articles_metadata.csv</i></font>

In [4]:
print("Le jeu de données df_articles_meta est composé de", df_articles_meta.shape[1], "colonnes ou attributs et ", df_articles_meta.shape[0],
      " lignes", )

Le jeu de données df_articles_meta est composé de 5 colonnes ou attributs et  364047  lignes


<font color = "cyan"><i> Apercu des permières lignes et colonnes de clicks_sample.csv</i></font>

In [5]:
print("Le jeu de données df_cliks_samp est composé de", df_cliks_samp.shape[1], "colonnes ou attributs et ", df_cliks_samp.shape[0],
      " lignes", )

Le jeu de données df_cliks_samp est composé de 12 colonnes ou attributs et  1883  lignes


<font color = "cyan"><i> Apercu des permières lignes et colonnes de clicks.csv</i></font>

In [6]:
print("Le jeu de données df_cliks est composé de", df_cliks.shape[1], "colonnes ou attributs et ", df_cliks.shape[0],
      " lignes", )

Le jeu de données df_cliks est composé de 12 colonnes ou attributs et  2569  lignes


<a id='data_size'></a>
### I.3) Entrevue du jeu de données
<font color = "cyan"><i> Apercu des permières lignes du jeu de données df_articles_meta</i></font>

In [7]:
df_articles_meta.head()

,article_id,category_id,created_at_ts,publisher_id,words_count
0,0,0,1513144419000,0,168
1,1,1,1405341936000,0,189
2,2,1,1408667706000,0,250
3,3,1,1408468313000,0,230
4,4,1,1407071171000,0,162


<font color = "cyan"><i> Apercu des permières lignes du jeu de données df_cliks_samp</i></font>

In [8]:
df_cliks_samp.head()

,user_id,session_id,session_start,session_size,click_article_id,click_timestamp,click_environment,click_deviceGroup,click_os,click_country,click_region,click_referrer_type
0,0,1506825423271737,1506825423000,2,157541,1506826828020,4,3,20,1,20,2
1,0,1506825423271737,1506825423000,2,68866,1506826858020,4,3,20,1,20,2
2,1,1506825426267738,1506825426000,2,235840,1506827017951,4,1,17,1,16,2
3,1,1506825426267738,1506825426000,2,96663,1506827047951,4,1,17,1,16,2
4,2,1506825435299739,1506825435000,2,119592,1506827090575,4,1,17,1,24,2


<font color = "cyan"><i> Apercu des permières lignes du jeu de données df_cliks</i></font>

In [9]:
df_cliks.head()

,user_id,session_id,session_start,session_size,click_article_id,click_timestamp,click_environment,click_deviceGroup,click_os,click_country,click_region,click_referrer_type
0,170835,1508207781378377,1508207781000,2,209122,1508207781268,4,3,2,1,25,2
1,170835,1508207781378377,1508207781000,2,289003,1508207811268,4,3,2,1,25,2
2,306892,1508207782230378,1508207782000,3,50644,1508208270611,4,4,20,1,25,2
3,306892,1508207782230378,1508207782000,3,211442,1508208475029,4,4,20,1,25,2
4,306892,1508207782230378,1508207782000,3,16346,1508208505029,4,4,20,1,25,2


<a id='data_def'></a>
## I.3) Types de données et leurs effectif
<font color = "cyan"><i> affichage des type et de leurs effectif df_articles_meta</i></font>

In [10]:
unsorted_df = pd.DataFrame({'colonne': df_articles_meta.dtypes.index,
                            'Types': df_articles_meta.dtypes.values,
                            'Effectif manquant': df_articles_meta.isna().sum().values,
                            'Effectif manquant %': ((df_articles_meta.isna().sum(axis=0) / df_articles_meta.shape[0]) * 100).astype(
                                float).values}).style.set_caption("Valeurs manquantes et leurs types df_articles_meta ")
unsorted_df

,colonne,Types,Effectif manquant,Effectif manquant %
0,article_id,int64,0,0.000000
1,category_id,int64,0,0.000000
2,created_at_ts,int64,0,0.000000
3,publisher_id,int64,0,0.000000
4,words_count,int64,0,0.000000


<font color = "cyan"><i> affichage des type et de leurs effectif df_cliks_samp</i></font>

In [11]:
unsorted_df = pd.DataFrame({'colonne': df_cliks_samp.dtypes.index,
                            'Types': df_cliks_samp.dtypes.values,
                            'Effectif manquant': df_cliks_samp.isna().sum().values,
                            'Effectif manquant %': ((df_cliks_samp.isna().sum(axis=0) / df_cliks_samp.shape[0]) * 100).astype(
                                float).values}).style.set_caption("Valeurs manquantes et leurs types df_cliks_samp ")
unsorted_df

,colonne,Types,Effectif manquant,Effectif manquant %
0,user_id,int64,0,0.000000
1,session_id,int64,0,0.000000
2,session_start,int64,0,0.000000
3,session_size,int64,0,0.000000
4,click_article_id,int64,0,0.000000
5,click_timestamp,int64,0,0.000000
6,click_environment,int64,0,0.000000
7,click_deviceGroup,int64,0,0.000000
8,click_os,int64,0,0.000000
9,click_country,int64,0,0.000000


<font color = "cyan"><i> affichage des type et de leurs effectif df_cliks</i></font>

In [12]:
unsorted_df = pd.DataFrame({'colonne': df_cliks.dtypes.index,
                            'Types': df_cliks.dtypes.values,
                            'Effectif manquant': df_cliks.isna().sum().values,
                            'Effectif manquant %': ((df_cliks.isna().sum(axis=0) / df_cliks.shape[0]) * 100).astype(
                                float).values}).style.set_caption("Valeurs manquantes et leurs types df_cliks ")
unsorted_df


,colonne,Types,Effectif manquant,Effectif manquant %
0,user_id,int64,0,0.000000
1,session_id,int64,0,0.000000
2,session_start,int64,0,0.000000
3,session_size,int64,0,0.000000
4,click_article_id,int64,0,0.000000
5,click_timestamp,int64,0,0.000000
6,click_environment,int64,0,0.000000
7,click_deviceGroup,int64,0,0.000000
8,click_os,int64,0,0.000000
9,click_country,int64,0,0.000000


><font color = "green"><i> Les fichiers ne contiennent aucunes valeurs manquante</i></font>


In [13]:
df = df_cliks.merge(df_articles_meta, left_on='click_article_id', right_on='article_id')
df = df[['user_id', 'article_id', 'category_id']]
df

,user_id,article_id,category_id
0,170835,209122,332
1,150849,209122,332
2,147014,209122,332
3,116929,209122,332
4,308699,209122,332
...,...,...,...
2564,322895,63746,133
2565,51099,98243,220
2566,130978,229190,375
2567,34979,331149,436


* Création de la matrice user-rating

In [14]:
series = df.groupby(['user_id', 'category_id']).size()

user_rating = series.to_frame()
user_rating = user_rating.reset_index()
user_rating.rename(columns = {0:'rating'}, inplace = True)
user_rating

,user_id,category_id,rating
0,8,26,1
1,8,375,1
2,77,43,1
3,77,428,1
4,111,99,1
...,...,...,...
2301,322894,297,1
2302,322895,133,1
2303,322895,418,1
2304,322896,26,1


In [15]:
data = user_rating['rating'].describe()

df_ser = pd.DataFrame({'Caractéristiques':data.index,
               'Valeurs':data.values
               })
df_ser.style.set_caption(f'Caractéristiques user_rating')

,Caractéristiques,Valeurs
0,count,2306.000000
1,mean,1.114050
2,std,0.397937
3,min,1.000000
4,25%,1.000000
5,50%,1.000000
6,75%,1.000000
7,max,6.000000


In [16]:
user_rating.rating.value_counts()

1    2090
2     183
3      23
4       7
5       2
6       1
Name: rating, dtype: int64

><font color = "cyan"><i> Création de la matrice **user_rating_s** pour le **fichier sample**</i></font>


In [17]:
df_s = df_cliks_samp.merge(df_articles_meta, left_on='click_article_id', right_on='article_id')
df_s = df_s[['user_id', 'article_id', 'category_id']]
df_s

,user_id,article_id,category_id
0,0,157541,281
1,20,157541,281
2,44,157541,281
3,45,157541,281
4,76,157541,281
...,...,...,...
1878,681,260988,395
1879,696,355863,448
1880,703,84094,174
1881,703,83472,174


In [18]:
series = df_s.groupby(['user_id', 'category_id']).size()

user_rating_s = series.to_frame()
user_rating_s = user_rating_s.reset_index()
user_rating_s.rename(columns = {0:'rating'}, inplace = True)
user_rating_s

,user_id,category_id,rating
0,0,136,1
1,0,281,1
2,1,209,1
3,1,375,1
4,2,26,1
...,...,...,...
1707,705,247,1
1708,705,412,1
1709,706,136,1
1710,706,209,1


In [19]:
user_rating_enough_s = user_rating_s.loc[user_rating_s.rating > 1]
reader = Reader()
data_s = Dataset.load_from_df(user_rating_enough_s[['user_id', 'category_id', 'rating']], reader)

trainset_s, testset_s = train_test_split(data_s, test_size=0.25)

><font color = "cyan"><i>Jeu de données pour la librairie **Implicit**</i></font>


In [20]:
df.columns

Index(['user_id', 'article_id', 'category_id'], dtype='object')

In [21]:

from sklearn.model_selection import train_test_split
train, cros_val = train_test_split(df, test_size = 0.2)

In [22]:
print('Splitted dataset into train set and cross validation set')
print('Train shape :', train.shape)
print('cros_val shape :', cros_val.shape)

Splitted dataset into train set and cross validation set
Train shape : (2055, 3)
cros_val shape : (514, 3)


## II Content-Based Filtering Model

![img](data/sys-rec_col.JPG )

> Le **filtrage basé sur le contenu** exploite la description des articles avec lesquels
 l’utilisateur a interagis afin de recommander des articles similaires.
 Cela ne dépend que des choix précédents de l’utilisateur, ce qui rend cette méthode robuste pour éviter le problème de démarrage à froid. On peut ainsi créer des profils d’ articles et des profils utilisateur.

> La méthode **TF-IDF** est la plus couramment utilisée pour convertir l' article en une structure vectorielle, où chaque mot est représenté par une position dans le vecteur, et la valeur mesure la pertinence d’un mot donné dans cet article.
 Comme tous les éléments seront représentés dans le même modèle d’espace vectoriel, il s’agit de calculer la similitude entre les articles par le **cosinus de l'angle** entre les différents vecteurs.



<font color = "grey"><i>Chargement de la matrice **embedding** des articles</i></font>


In [23]:
with open(r"data/articles_embeddings.pickle", "rb") as input_file:
    pickle_articles_emb = cPickle.load(input_file)

pickle_articles_emb.shape

(364047, 250)

<font color = "cyan"><i>récupération de **n articles similaires** pour un utilisateur (**par user_id**)</i></font>




In [24]:
#nb of world
def getnArticles(pickle_articles_emb, n, userId):

    pickle_articles_emb_bis = pickle_articles_emb
    # get all articles read by user
    var = df_cliks.loc[df_cliks['user_id']==userId]['click_article_id'].tolist()

    # chose randomly one
    value = randint(0, len(var))

    # delete all read articles except the selected one( we do not want to offer
    # user to read something he already read)
    for i in range(0, len(var)):
        if i != value:
            pickle_articles_emb_bis = np.delete(pickle_articles_emb_bis, [i], 0)

    arr = []

    # delete selected article in the new matrix
    f = np.delete(pickle_articles_emb_bis, [value], 0)

    # get n articles the most similar to the selected one
    for i in range(0, n):
        distances = distance.cdist([pickle_articles_emb_bis[value]], f, "cosine")[0]
        min_index = np.argmin(distances)
        f = np.delete(f, [min_index], 0)
    # find corresponding matrix in original martix
        result = np.where(pickle_articles_emb==f[min_index])
        arr.append(result[0][0])

    return arr

arr = getnArticles(pickle_articles_emb, 10, 0)
arr

[47678, 77966, 34030, 10345, 32934, 27638, 77297, 62907, 24634, 1803]

* On ajouter **le nombre de mots** de chaque article dans la matrice **embedding**

In [25]:
d = pickle_articles_emb
max = df_articles_meta['words_count'].max()
df_articles_meta['words_count'] = df_articles_meta['words_count'].apply(lambda x: x/max)

d = np.append(d, np.reshape(df_articles_meta['words_count'].to_numpy(),
                            newshape=(df_articles_meta['words_count'].shape[0], 1)),
              axis=1)

d.shape

(364047, 251)

In [26]:
arr = getnArticles(d, 10, 0)

<font color = "green"><i> Voici la liste des 10 articles similaires lus et vus par l'utilisateur 0.</i></font>

><font color = "green"><i> Dans la recommandation par le contenu, nous avons proposé parmi tous les articles, ceux qui sont similaires aux articles sur lesquels l'utilisateur a cliqués.</i></font>

In [27]:
arr

[629, 286, 306, 61, 48, 144, 58, 78, 840, 15]

# III) Collaborative Filtering Model
![img](data/sys-rec_u.JPG )

> <font color = "green"> Le **filtrage basé sur la collaboration** fait des prédictions automatiques (filtrage) sur les intérêts d’un utilisateur en collectant les préférences ou les informations de goût de nombreux utilisateurs (collaboration). L’hypothèse sous-jacente de l’approche de filtrage collaboratif est que si une personne A a la même opinion qu’une personne B sur un ensemble d’éléments, A est plus susceptible d’avoir l’opinion de B pour un élément donné que celle d’une personne choisie au hasard.
</i></font>


 <font color = "green"> On a 2 approches d'implémentation du **Collaborative Filtering**:<i>
* une basée sur la mémoire : cette approche utilise la mémoire des interactions des utilisateurs précédents pour calculer les similitudes des utilisateurs en fonction des éléments avec lesquels ils ont interagis (approche basée sur l’utilisateur) ou calculer les similitudes des éléments en fonction des utilisateurs qui ont interagi avec eux (approche basée sur les éléments).
* basé sur un modèle : les modèles sont développés à l’aide de différents algorithmes d’apprentissage automatique pour recommander des éléments aux utilisateurs. Il existe de nombreux algorithmes CF basés sur des modèles, tels que les réseaux de neurones, les réseaux bayésiens, les modèles de clustering et les modèles de facteurs latents tels que la décomposition de valeur singulière (SVD) et l’analyse sémantique latente probabiliste
</i></font>

## Création d'un rating pour chaque catégorie d'article vue par chaque utilisateur



In [28]:
user_rating.rating.describe

data = user_rating['rating'].describe()

df_ser = pd.DataFrame({'Caractéristiques':data.index,
               'Valeurs':data.values
               })
df_ser.style.set_caption(f'Caractéristiques rating')

,Caractéristiques,Valeurs
0,count,2306.000000
1,mean,1.114050
2,std,0.397937
3,min,1.000000
4,25%,1.000000
5,50%,1.000000
6,75%,1.000000
7,max,6.000000


In [29]:
user_rating.loc[user_rating.rating > 0]

,user_id,category_id,rating
0,8,26,1
1,8,375,1
2,77,43,1
3,77,428,1
4,111,99,1
...,...,...,...
2301,322894,297,1
2302,322895,133,1
2303,322895,418,1
2304,322896,26,1


In [30]:
user_rating_enough = user_rating.loc[user_rating.rating > 0]
#print('On a sélectionné ', len(user_rating_enough), ' interactions.')

><font color = "green"><i> 2306 ont été selectionné </i></font>

In [31]:
from surprise.model_selection import train_test_split

reader = Reader()
data = Dataset.load_from_df(user_rating_enough[['user_id', 'category_id', 'rating']], reader)

trainset, testset = train_test_split(data, test_size=0.25)
print('Test set lenght :', len(testset))
print('Train set lenght :', len(user_rating_enough) - len(testset))

Test set lenght : 577
Train set lenght : 1729


##  **k-NN** et optimisation des paramètres du modèle

><font color = "green"><i> Entrainement du modèle et recherche des paramètres optimaux </i></font>

In [32]:
from surprise import KNNWithMeans

sim_options = {
    "name": ["cosine"],
    "min_support": [3, 4, 5],
    "user_based": [False, True],
}

param_grid = {"sim_options": sim_options, "k": [5, 10], "min_k": [2, 3]}

gs = GridSearchCV(KNNWithMeans, param_grid, measures=["rmse", "mae"], cv=3)
gs.fit(data)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing th

* Performance du modèle **KNNWithMeans**

In [33]:
print(gs.best_score["rmse"])
print(gs.best_params["rmse"])

0.40579805566111005
{'sim_options': {'name': 'cosine', 'min_support': 4, 'user_based': False}, 'k': 5, 'min_k': 3}


In [34]:
algo_knn = KNNWithMeans(k=10, min_k=3)
algo_knn.fit(trainset)

cross_validate(algo_knn, data, measures=['RMSE', 'MAE'], cv=3, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNWithMeans on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.5011  0.4324  0.4908  0.4748  0.0302  
MAE (testset)     0.2041  0.1828  0.2026  0.1965  0.0097  
Fit time          0.06    0.05    0.05    0.05    0.01    
Test time         0.05    0.06    0.05    0.05    0.01    


{'test_rmse': array([0.50105938, 0.43243373, 0.49081723]),
 'test_mae': array([0.20412831, 0.18283452, 0.20261837]),
 'fit_time': (0.06099867820739746, 0.04600071907043457, 0.04699873924255371),
 'test_time': (0.04700064659118652, 0.06200146675109863, 0.05100059509277344)}

In [35]:
predictions_knn = algo_knn.test(testset)
accuracy.rmse(predictions_knn)

RMSE: 0.2951


0.2950882470201359


<font color = "cyan"><i>Sauvegarde du modèle et récupération des 10 meilleurs des articles pour chaque utilisateur</i></font>

In [36]:
from collections import defaultdict


def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n



In [37]:
top_10_knn = get_top_n(predictions_knn, n=10)

with open("top_10_knn.txt", "wb") as fp:
    pickle.dump(top_10_knn, fp)

file_model = open('algo_knn.pickle', 'wb')
pickle.dump(algo_knn, file_model)
file_model.close()

## **SVD** <i>Singular Value Decomposition</i> et optimisation des paramètres du modèle


><font color = "green"><i>Décomposition en valeurs singulières est un algorithme de factorisation de matrice les plus utilisés. L'algorithme **SVD** est efficace car il permet de trouver des variables latentes qui lient les utilisateurs et les articles
</i></font>




<font color = "cyan"><i>Entrainement du modèle et recherche des paramètres optimaux</i></font>

In [38]:
param_grid = {"n_epochs": [5, 10],
              "lr_all": [0.002, 0.005],
              "reg_all": [0.4, 0.6]
}

gs = GridSearchCV(SVD, param_grid, measures=["rmse", "mae"], cv=3)

gs.fit(data)

* Performance du modèle **SVD**

In [39]:
print(gs.best_score["rmse"])
print(gs.best_params["rmse"])

0.39262805798229133
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}


In [40]:
algo_svd = SVD()
algo_svd.fit(trainset)

cross_validate(algo_svd, data, measures=['RMSE', 'MAE'], cv=3, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.4470  0.3899  0.3474  0.3948  0.0408  
MAE (testset)     0.2075  0.1917  0.1840  0.1944  0.0098  
Fit time          0.12    0.12    0.12    0.12    0.00    
Test time         0.01    0.01    0.01    0.01    0.00    


{'test_rmse': array([0.44698428, 0.38992476, 0.34740025]),
 'test_mae': array([0.20747006, 0.19172768, 0.18402409]),
 'fit_time': (0.11899709701538086, 0.12499332427978516, 0.11700034141540527),
 'test_time': (0.00799870491027832,
  0.008001089096069336,
  0.007999897003173828)}

In [41]:
predictions_svd = algo_svd.test(testset)
accuracy.rmse(predictions_svd)

RMSE: 0.3054


0.3053826159683244


<font color = "cyan"><i> Sauvegarde du modèle et récupération du top 10 des articles pour chaque utilisateur</i></font>

In [42]:
top_10_svd = get_top_n(predictions_svd, n=10)

with open("top_10_svd.txt", "wb") as fp:
    pickle.dump(top_10_svd, fp)

file_model = open('algo_svd.pickle', 'wb')
pickle.dump(algo_svd, file_model)
file_model.close()


# IV. Librairie **Implicit**



<font color = "green"><i> L'algorithme **AlternatingLeastSquares** permet la factorisation de matrice dans le cas du Collaborative Filtering implicite.</i></font>

In [43]:
import scipy.sparse as sparse
import implicit

user_categories = sparse.csr_matrix((user_rating_enough['rating'].astype(float),
                                   (user_rating_enough['user_id'],
                                    user_rating_enough['category_id'])))
category_users = sparse.csr_matrix((user_rating_enough['rating'].astype(float),
                                (user_rating_enough['category_id'],
                                 user_rating_enough['user_id'])))

alpha = 40
train_c = (category_users*alpha).astype('double')

In [44]:
model = implicit.als.AlternatingLeastSquares(factors=500, iterations=10)

model.fit(train_c)

  0%|          | 0/10 [00:00<?, ?it/s]

In [45]:
import csv

fields = 'user_id', 'article_list'
filename = 'rec_train.csv'


df_recommendations= pd.DataFrame(columns=['user_id', 'article_list'])

In [46]:

userId = user_rating_enough['user_id'].values.tolist()
i=0
for user in userId:

    scores = []
    items = []
    results = []
    dict={}
    results.append(user)
    recommendations = model.recommend(user, user_categories, N = 5)
    for item in recommendations:
        ids, score = item
        scores.append(score)
        items.append(ids)
    results.append(items)

    if(user not in df_recommendations['user_id'].values):

        dict={'user_id': user,
            'article_list':items}
        df_recommendations = df_recommendations.append(dict, ignore_index = True)


In [47]:
df_recommendations.to_csv('df_recommendations.csv',index= False)


# V. Ajout de nouveau utilisateur ou d'utilisateur sans contenu



><font color = "green"><i> Tous les utilisateurs qui n'on pas de contenu consulté ou les nouveaux utilisateurs se verrons attribués
des articles de référence. Les articles de référence sont le top 10 des articles recommcomendé a tout le monde </i></font>



In [48]:
list_all_recom = []
for row in range (len(df_recommendations['article_list'])):
   list_all_recom = list_all_recom + df_recommendations['article_list'][row]

In [49]:
article = pd.Index(list_all_recom,name='article' )
article.value_counts().head(10)

281    512
375    500
43     493
340    465
99     405
331    338
136    315
354    275
26     190
412     78
Name: article, dtype: int64

<font color = "green"><i> top 10 des articles les plus recommandés</i></font>

**281 375 340 43 99 136 331 354 26 332**

In [50]:
article.value_counts().head(10).index



Int64Index([281, 375, 43, 340, 99, 331, 136, 354, 26, 412], dtype='int64')

><font color = "green"><i>L'ajout de nouvelle article se fera a la suite de dfarticle metadata </i></font>

